### Sensor Network Simulation 
#### *Using consensus-based filters*

Set up target & sensor network -

In [3]:
import os
import settings.input
import settings.initialize

# Input (.json) files containing target, network
simulation_file = os.path.join("settings","json_files", "simulation.json")
sensor_network_file = os.path.join("settings","json_files", "default-sensor-network.json")

# Read input files
raw_data = settings.input.read_configs([simulation_file, sensor_network_file])
raw_data["network"]

{'indexing_style': 'matlab',
 'adjacency': [[1, 0, 1, 1, 0],
  [0, 1, 1, 0, 1],
  [1, 1, 1, 1, 1],
  [1, 0, 1, 1, 0],
  [0, 1, 1, 0, 1]],
 'observation_matrices': {'default': [[1, 0], [0, 1]]},
 'noise_covariances': {'default': [[5, 0], [0, 5]]}}

In [4]:
input_data  = settings.initialize.do_everything(raw_data)

In [8]:
import sim.target
import sim.network
import sim.history

input_data["scheme"] = "OMVF"
input_data["network"]["noise_covariances"]["3"] = [[300, 0], [0, 300]]
target = sim.target.create(input_data)
network = sim.network.create(input_data)
omvf_h = sim.history.create(network)
sim_history = omvf_h

In [9]:
for t in range(50):

    sim_history.add_target(target.x)

    # Simulation...
    network.make_measurements(target.x)

    network.share_info_with_neighbors()
    network.get_info_about_target(target)

    network.do_estimations()
    sim_history.add_estimates(network)

    target.update()

# sim_history.plot_xy(target=False, estimates_of="3", measurements_of="2")

In [ ]:
# sim_history.plot_timeseries(ErrCov_of="2")
# plt.plot(plot_x, plot_y)

In [ ]:
# vars(network.sensors["3"])

In [ ]:
from matplotlib import pyplot as plt
import numpy.linalg as la
FIGURE_SIZE = (7, 7)
ErrCov_of = "3"

plt.rcParams["figure.figsize"] = FIGURE_SIZE
y_vals = [la.norm(_matrix, 'fro') for _matrix in kcf_h.sensors[ErrCov_of]["ErrCov"]]
plt.plot(y_vals, label=f"KCF")
y_vals = [la.norm(_matrix, 'fro') for _matrix in icf_h.sensors[ErrCov_of]["ErrCov"]]
plt.plot(y_vals, label=f"ICF (2013)")
y_vals = [la.norm(_matrix, 'fro') for _matrix in omvf_h.sensors[ErrCov_of]["ErrCov"]]
plt.plot(y_vals, label=f"Optimal MVF")


plt.xlabel("Iteration")
plt.ylabel("Err Cov. (Frob. Norm)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
input_data

In [ ]:
# from sim.noise import Noise
# from matplotlib.pyplot import plot, show
# x = []
# y = []
# for i in range(200):
#     a = Noise([[0.000002,0],[0,0.000002]]).sample()
#     x.append(a[0])
#     y.append(a[1])
    
# plot(x,y, 'ro')
# show()